In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


In [2]:
# path to the AIS data CSV file 
file_path = '/Users/macbokpro/Desktop/2025/Port-related emission project/Emission Inventory/coding/vessels at anchorage state/AIS data/AIS_2022_05.csv'
anchorage_shapefile = '/Users/macbokpro/Desktop/2025/Port-related emission project/Emission Inventory/coding/vessels at anchorage state/anchorage area/AnchorageAreas.shp'

# Vessel characteristics
main_engine_power = 8130  # kW
aux_engine_power = 740  # kW
max_speed = 14.5  # knots
ef_main_engine = 14.4  # g/kWh (NOx emission factor for main engine)
ef_aux_engine = 10.5  # g/kWh (NOx emission factor for auxiliary engine)

# Load the AIS data
ais_data = pd.read_csv(file_path)

/var/folders/99/pbr5mvn519lclv42n03yb1hr0000gn/T/ipykernel_84305/548620524.py:13: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  ais_data = pd.read_csv(file_path)


In [3]:
# Filter data for vessel IMO9922249 and the date 2022-05-06
vessel_imo = 'IMO9922249'
ais_data['Timestamp'] = pd.to_datetime(ais_data['BaseDateTime'])
ais_data_filtered = ais_data[(ais_data['IMO'] == vessel_imo) & (ais_data['Timestamp'].dt.date == pd.to_datetime('2022-05-06').date())]


In [5]:
# Load Anchorage area shapefile
anchorage_areas = gpd.read_file(anchorage_shapefile)

# Convert AIS data to GeoDataFrame for spatial analysis
geometry = [Point(xy) for xy in zip(ais_data_filtered['LON'], ais_data_filtered['LAT'])]
ais_gdf = gpd.GeoDataFrame(ais_data_filtered, geometry=geometry, crs=anchorage_areas.crs)

# Check if the vessel is within the anchorage area
ais_gdf['in_anchorage'] = ais_gdf.within(anchorage_areas.unary_union)

/var/folders/99/pbr5mvn519lclv42n03yb1hr0000gn/T/ipykernel_84305/1816043514.py:9: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  ais_gdf['in_anchorage'] = ais_gdf.within(anchorage_areas.unary_union)


In [6]:
# Function to classify operational mode
def classify_mode(row):
    speed = row['SOG']  # Speed Over Ground in AIS data
    in_anchorage = row['in_anchorage']
    
    if speed == 0:
        if in_anchorage:
            return 'Anchorage Hotelling'
        else:
            return 'Berth Hotelling'
    elif speed < 1 and in_anchorage:
        return 'Anchorage Hotelling'
    elif speed < 3:
        return 'Maneuvering'
    else:
        return 'Transit'

# will refine classification later

In [7]:
# Apply classification
ais_gdf['Operational Mode'] = ais_gdf.apply(classify_mode, axis=1)

In [8]:
# Calculate the duration between consecutive timestamps
ais_gdf['Next Timestamp'] = ais_gdf['Timestamp'].shift(-1)
ais_gdf['Duration (hours)'] = (ais_gdf['Next Timestamp'] - ais_gdf['Timestamp']).dt.total_seconds() / 3600

# Fill NaN for the last row (as there's no next timestamp) with 0
ais_gdf['Duration (hours)'] = ais_gdf['Duration (hours)'].fillna(0)

In [9]:
# Apply Propeller Law to calculate main engine operating power based on SOG and vessel characteristics
ais_gdf['Main Engine Power (kW)'] = ((ais_gdf['SOG'] / max_speed) ** 3) * main_engine_power

In [10]:
# Calculate NOx emissions for each mode
def calculate_emissions(row):
    mode = row['Operational Mode']
    duration = row['Duration (hours)']
    
    if mode == 'Transit':
        # Only main engine runs
        emissions_main = row['Main Engine Power (kW)'] * duration * ef_main_engine
        return emissions_main
    
    elif mode == 'Maneuvering':
        # Both main engine and auxiliary engine run
        emissions_main = row['Main Engine Power (kW)'] * duration * ef_main_engine
        emissions_aux = aux_engine_power * duration * ef_aux_engine
        return emissions_main + emissions_aux
    
    elif mode in ['Anchorage Hotelling', 'Berth Hotelling']:
        # Only auxiliary engine runs
        emissions_aux = aux_engine_power * duration * ef_aux_engine
        return emissions_aux
    
    return 0


In [11]:
# Apply the emissions calculation
ais_gdf['NOx Emissions (g)'] = ais_gdf.apply(calculate_emissions, axis=1)

In [12]:
# Group by the operational mode and sum the emissions for each mode
emissions_by_mode = ais_gdf.groupby('Operational Mode')['NOx Emissions (g)'].sum()


In [14]:
# Display the AIS data with emissions calculated and total emissions by mode
print("Filtered AIS Data for IMO 9922249 on 2022-05-06 with NOx Emissions:")
print(ais_gdf[['Timestamp', 'SOG', 'LON', 'LAT', 'Operational Mode', 'Duration (hours)', 'Main Engine Power (kW)', 'NOx Emissions (g)']])



Filtered AIS Data for IMO 9922249 on 2022-05-06 with NOx Emissions:
                 Timestamp   SOG        LON       LAT Operational Mode  \
654088 2022-05-06 15:27:23  11.1 -119.49961  34.09286          Transit   
654091 2022-05-06 15:28:32  11.2 -119.49543  34.09191          Transit   
654093 2022-05-06 15:29:43  11.2 -119.49113  34.09092          Transit   
654096 2022-05-06 15:30:52  11.1 -119.48757  34.09011          Transit   
654098 2022-05-06 15:31:53  11.2 -119.48337  34.08915          Transit   
...                    ...   ...        ...       ...              ...   
655068 2022-05-06 23:55:51   0.3 -118.21487  33.71055      Maneuvering   
655071 2022-05-06 23:57:07  14.4 -118.21486  33.71056          Transit   
655075 2022-05-06 23:58:17   0.5 -118.21487  33.71057      Maneuvering   
655078 2022-05-06 23:59:28   0.2 -118.21487  33.71058      Maneuvering   
655079 2022-05-06 23:59:37   2.0 -118.21065  33.71169      Maneuvering   

        Duration (hours)  Main Engine Power

In [15]:
print("\nTotal NOx Emissions by Operational Mode:")
print(emissions_by_mode)


Total NOx Emissions by Operational Mode:
Operational Mode
Berth Hotelling      4187.166667
Maneuvering          6769.467876
Transit            346423.583690
Name: NOx Emissions (g), dtype: float64
